### Referencias modelo gravitacional

https://es.wikipedia.org/wiki/Modelo_de_gravedad_del_comercio  
https://www.nature.com/articles/s41467-021-26752-4

In [1]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)

### Datos Estudiantes

In [3]:
parvulario=pd.read_csv('parvulario.csv')
escolar=pd.read_csv('escolar.csv')
superior=pd.read_csv('superior.csv')    

In [5]:
educacion=pd.concat([parvulario,escolar,superior])

In [6]:
educacion['comuna']=educacion['comuna'].str.title()
educacion['comuna']=educacion['comuna'].str.strip().str.lower()
educacion=educacion.rename(columns={'ano':'año'})
educacion=educacion.groupby(['comuna', 'año']).agg({'matriculas': 'sum'}).reset_index()

### Datos Laborales

In [7]:
laborales=pd.read_excel('datos_laborales.xls')

In [8]:
laborales_melt=laborales.melt(id_vars='Unidad territorial').query("variable !=' Variable' ").rename(
    columns={'variable':'año','value':'trabajadores','Unidad territorial':'comuna'})

In [9]:
laborales_melt['comuna']=laborales_melt['comuna'].str.strip().str.lower()

In [10]:
laborales_melt['año']=laborales_melt['año'].str.strip().astype(int)

#### Prediccion para el 2024

In [11]:
from sklearn.linear_model import LinearRegression

predicciones = {}
comunas = laborales_melt['comuna'].unique()

for comuna in comunas:
    data_comuna = laborales_melt[laborales_melt['comuna'] == comuna]
    X = data_comuna['año'].values.reshape(-1, 1)
    y = data_comuna['trabajadores'].values
    
    model = LinearRegression()
    model.fit(X, y)
    
    prediccion_2024 = model.predict([[2024]])[0]
    predicciones[comuna] = max(0, prediccion_2024)  

resultados = pd.DataFrame({
    'comuna': list(predicciones.keys()),
    'año': 2024 ,
    'trabajadores': list(predicciones.values())
})

resultados = resultados.sort_values('comuna').reset_index(drop=True)


In [12]:
laborales_melt=pd.concat([laborales_melt,resultados])

### Datos Poblacionales

In [14]:
poblacion=pd.read_excel('estimaciones-y-proyecciones-2002-2035-comunas.xlsx',sheet_name='poblacion_total')

In [15]:
poblacion_melt=poblacion.melt(id_vars='Comuna').query("variable !=2025 ").rename(columns={'variable':'año','value':'poblacion','Comuna':'comuna'})

In [16]:
poblacion_melt['comuna']=poblacion_melt['comuna'].str.strip().str.lower()

### Merge Datasets

In [17]:
df=poblacion_melt.merge(laborales_melt, on=['comuna', 'año'], how='left'). \
merge(educacion, on=['comuna', 'año'], how='left')

#### Considerar desde 2020 en adelante

In [18]:
df=df.query('año >= 2020')

### Leer Comunas

In [21]:
import geopandas as gpd

In [ ]:
comunas=gpd.read_file('datos/Comunas/comunas.shp')

In [25]:
comunas_rm=comunas.query("Region=='Región Metropolitana de Santiago'")

### Generacion Matriz de Movilidad

In [26]:
import geopandas as gpd
import numpy as np
from scipy.spatial import distance_matrix


comunas_rm['geometry'] = comunas_rm.geometry.centroid
comunas_rm['Comuna'] = comunas_rm['Comuna'].str.strip().str.lower()

coords = np.array([(point.x, point.y) for point in comunas_rm['geometry']])

dist_matrix = distance_matrix(coords, coords)

comunas = comunas_rm['Comuna'].tolist()  
dist_df = pd.DataFrame(dist_matrix, index=comunas, columns=comunas)

beta = 2  
K = 1 

years = df['año'].unique()
flow_matrices = {}

for year in years:
    df_year = df[df['año'] == year].set_index('comuna')
    
    flow_matrix = np.zeros((len(comunas), len(comunas)))
    
    for i, comuna_i in enumerate(comunas):
        for j, comuna_j in enumerate(comunas):
            if i == j:
                flow_matrix[i, j] = 0 
            else:
                P_i = df_year.loc[comuna_i, 'poblacion']  
                A_j = df_year.loc[comuna_j, 'trabajadores']  
                D_ij = dist_df.loc[comuna_i, comuna_j] 
                
        
                flow_matrix[i, j] = K * (P_i * A_j) / (D_ij ** beta)
    
    flow_matrices[year] = pd.DataFrame(flow_matrix, index=comunas, columns=comunas)

/Users/mespinoza/miniforge3/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


### Normalizacion de matriz con softmax

In [27]:
def funcion_normalizacion(matriz):
    max_vals = np.max(matriz, axis=1, keepdims=True)
    exp_matriz = np.exp(matriz - max_vals)
    sum_exp = np.sum(matriz, axis=1, keepdims=True)
    softmax_matriz = matriz / sum_exp
    return softmax_matriz

In [28]:
matrices_norm = {}

for year, matrix in flow_matrices.items():
    matrix_values = matrix.values
    normalized_matrix = funcion_normalizacion(matrix_values)
    normalized_df = pd.DataFrame(normalized_matrix, index=matrix.index, columns=matrix.columns)
    matrices_norm[year] = normalized_df


In [29]:
matrices_norm

{2020:                      san joaquín  san miguel  san ramón  independencia  \
 san joaquín             0.000000    0.139560   0.003696       0.004882   
 san miguel              0.080974    0.000000   0.004921       0.005713   
 san ramón               0.027604    0.063350   0.000000       0.004806   
 independencia           0.002817    0.005681   0.000371       0.000000   
 la cisterna             0.027607    0.101803   0.036416       0.005638   
 peñalolén               0.007559    0.009736   0.001275       0.004096   
 providencia             0.006249    0.009311   0.000613       0.016234   
 la reina                0.003534    0.004746   0.000499       0.003123   
 calera de tango         0.007004    0.016184   0.003076       0.005934   
 colina                  0.004567    0.008641   0.001019       0.010031   
 santiago                0.016006    0.034947   0.001384       0.032340   
 lampa                   0.004654    0.009680   0.001107       0.011384   
 pirque            